## 띄어쓰기 보정을 적용한 데이터셋을 출력하는 곳

made by eyeol

**related work**
- ./data/analysis/eyeol_analysis.ipynb
- ./data/analysis/token_tracker.ipynb

### 0. Spacing을 선택한 이유

#### - 1) rtt와 sampled의 차이 발견

Confusion Matrix 분석 후, rtt와 sampled의 생태가 다르다고 느낌</br>

rtt와 sampled 데이터를 관찰해보니</br> 
sentence_1과 sentence_2의 양상이 상당히 달랐음</br>

#### - 2) rtt 데이터의 특징 발견

rtt는 두 문장이 상당히 유사하지만, 띄어쓰기 적용이 동일하진 않았다</br> 

토크나이징 결과를 관찰해보니, 주요한 단어들이 띄어쓰기 차이로 다르게 토큰화되는 것을 확인 </br>

```plaintext
'[CLS]', '앗', '제', '##가', '접근', '##권', '##한', '##이', '없', '##다고'
'[CLS]', '앗', '제', '##가', '접근', '권한', '##이', '없', '##다고'

'권한' 이라는 단어가 다르게 토크나이징됨
```

그래서 rtt 데이터에 띄어쓰기 보정을 하면 성능 향상이 될 것 같았음 <br>
같은 단어 구성이 다르게 토큰화되던게, 띄어쓰기를 고치면 동일하게 토큰화될테니까 <br>



sampled 중에서 target이 4점 이상인 데이터들은 rtt와 비슷한 양상이라  </br>
해당 데이터들에도 띄어쓰기 적용해보면 좋겠다고 생각함


#### - 3) 실험 순서 결정
일단 rtt에 spacing을 먼저 해보고, 성능 향상이 있으면 </br>
sampled 중 4점 이상 데이터들에도 적용하자고 결정

### 1. Spacing 적용

In [2]:
import pandas as pd
from kiwipiepy import Kiwi

In [3]:
kiwi = Kiwi()

def correct_spacing(text):
    return kiwi.space(text)

In [4]:
train_data = pd.read_csv('../raw/train.csv')

#### train data 중에서 rtt만 필터링 후 spacing 적용

In [5]:
train_data_filtered = train_data[train_data['source'].str.endswith('rtt')]

In [6]:
train_data_filtered['sentence_1'] = train_data_filtered['sentence_1'].apply(lambda x: correct_spacing(x))
train_data_filtered['sentence_2'] = train_data_filtered['sentence_2'].apply(lambda x: correct_spacing(x))

C:\Users\eyeol\AppData\Local\Temp\ipykernel_15804\12992353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_filtered['sentence_1'] = train_data_filtered['sentence_1'].apply(lambda x: correct_spacing(x))
C:\Users\eyeol\AppData\Local\Temp\ipykernel_15804\12992353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_filtered['sentence_2'] = train_data_filtered['sentence_2'].apply(lambda x: correct_spacing(x))


In [7]:
train_data.update(train_data_filtered)

In [10]:
train_data.to_csv('./v1/rtt_spaced_train.csv', index=False)

그런데 해당 데이터셋을 사용했을 때, 성능이 오르기는커녕 소폭 감소함..
그래서 sampled에 대한 spacing은 일단 보류했음